<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Steam games dataset 2024
</div>

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
<strong>Extraccion de informacion</strong>

<p>
Recuperdor de información, para saber  que tipo de contenido se desea como resumenes, detallesm reseñas, edades
</p>
</div>

In [9]:
import spacy_transformers
import pandas as pd
import re
import spacy
import textacy
import visualise_spacy_tree
import glob
from spacy.matcher import Matcher 
import os
import sys
import nltk
import en_core_web_trf
import re
from pathlib import Path
from pprint import pprint
from sklearn_crfsuite import CRF
from nltk.corpus import opinion_lexicon
from spacy.matcher import Matcher
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from sklearn_crfsuite import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

import numpy as np
from collections import defaultdict
from collections import Counter

In [10]:
pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows=100

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos esta linea ya que teniamos que definir cada columna con su tipo de dato para no tener problemas luego
</div>

In [11]:
dtype_dict = {
    'AppID': 'int64',
    'name': 'object',
    'release_date': 'object',
    'required_age': 'object',
    'price': 'float64',
    'dlc_count': 'float64',
    'detailed_description': 'object',
    'about_the_game': 'object',
    'short_description': 'object',
    'reviews': 'object',
    'header_image': 'object',
    'website': 'object',
    'support_url': 'object',
    'support_email': 'object',
    'windows': 'object',
    'mac': 'object',
    'linux': 'object',
    'metacritic_score': 'object',
    'metacritic_url': 'object',
    'achievements': 'object',
    'recommendations': 'object',
    'notes': 'object',
    'supported_languages': 'object',
    'full_audio_languages': 'object',
    'packages': 'object',
    
    'Unnamed: 87': 'float64',
    'Unnamed: 88': 'float64',
    'Unnamed: 89': 'float64'
}


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Leemos el dataset 
</div>

In [12]:
df = pd.read_csv('games_may2024_cleaned.csv',  dtype=dtype_dict, low_memory=False)

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Elegimos las columnas que queremos extraer su informacion
</div>

In [13]:
df = df[[ "about_the_game", "short_description","genres", "supported_languages"]]

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos estas 2 celdas ya que no queremos los NAN en cada fila de cada columna
</div>

In [14]:
df_cleaned = df.dropna()

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Podemos ver el df sin los NAN en cada fila de cada columna 
</div>

In [15]:
df_cleaned

,about_the_game,short_description,genres,supported_languages
0,for over two decades counterstrike has offered an elite competitive experience one shaped by millions of players from across the globe and now the next chapter in the cs story is about to begin this is counterstrike 2 a free upgrade to csgo counterstrike 2 marks the largest technical leap in counterstrike’s history built on the source 2 engine counterstrike 2 is modernized with realistic physicallybased rendering state of the art networking and upgraded community workshop tools in addition to the classic objectivefocused gameplay that counterstrike pioneered in 1999 counterstrike 2 features allnew cs ratings with the updated premier mode global and regional leaderboards upgraded and overhauled maps gamechanging dynamic smoke grenades tickrateindependent gameplay redesigned visual effects and audio all items from csgo moving forward to cs2,for over two decades counterstrike has offered an elite competitive experience one shaped by millions of players from across the globe and now the next chapter in the cs story is about to begin this is counterstrike 2,action free to play,czech danish dutch english finnish french german hungarian italian japanese korean norwegian polish portuguese portugal portuguese brazil romanian russian simplified chinese spanish spain swedish thai traditional chinese turkish bulgarian ukrainian greek spanish latin america vietnamese
1,land loot survive play pubg battlegrounds for free land on strategic locations loot weapons and supplies and survive to become the last team standing across various diverse battlegrounds squad up and join the battlegrounds for the original battle royale experience that only pubg battlegrounds can offer this content download will also provide access to the battlegrounds test server which requires a separate download to play optional ingame purchases available,play pubg battlegrounds for free land on strategic locations loot weapons and supplies and survive to become the last team standing across various diverse battlegrounds squad up and join the battlegrounds for the original battle royale experience that only pubg battlegrounds can offer,action adventure massively multiplayer free to play,english korean simplified chinese french german spanish spain arabic japanese polish portuguese portugal russian turkish thai italian portuguese brazil traditional chinese ukrainian
2,the mostplayed game on steam every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a constant evolution of gameplay features and heroes dota 2 has truly taken on a life of its own one battlefield infinite possibilities when it comes to diversity of heroes abilities and powerful items dota boasts an endless arrayno two games are the same any hero can fill multiple roles and theres an abundance of items to help meet the needs of each game dota doesnt provide limitations on how to play it empowers you to express your own style all heroes are free competitive balance is dotas crown jewel and to ensure everyone is playing on an even field the core content of the gamelike the vast pool of heroesis available to all players fans can collect cosmetics for heroes and fun addons for the world they inhabit but everything you need to play is already included before you join your first match bring your friends and party up dota is deep and constantly evolving but its never too late to join learn the ropes playing coop vs bots sharpen your skills in the hero demo mode jump into the behavior and skillbased matchmaking system that ensures youll be matched with the right players each game,every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos la limpieza profunda
</div>

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Instalamos la libreria de emoji para poder eliminar cada emoji que aparezca en el dataset
</div>

In [16]:
#%pip install emoji

In [17]:
import re

def clean_df(df):
    if df is None:
        return None
    df_copy = df.copy()
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002600-\U000026FF"  # Miscellaneous Symbols
                               u"\U00002700-\U000027BF"  # Dingbats
                               "]+", flags=re.UNICODE)
    
    # Define the replacement patterns
    replace_patterns = [
        ('•', False),
        ('◆', False),
        ('™', False),
        ('®', False),
        ('©', False),
        ('█', False),
        ('▒', False),
        ('__', False),
        ('……', False),
        ('†', False),
        ('※', False),
        ('', False),
        ('£', False),
        ('⊰⊱', False),
        ('๑̀́و', False),
        ("'", False),
        ('...', False),
        ('-', False),
        ('—', False),
        ('_', False),
        ('___________', False),
        (r'[,.:;=()*+&^%$#@!{}~\[\]<>?/|\\\-\'"«»–“”]', True)
    ]
    
    for columna in df_copy.columns:
        # Apply emoji pattern removal
        df_copy[columna] = df_copy[columna].astype(str).apply(lambda x: emoji_pattern.sub(r'', x))
        
        # Apply all replacement patterns
        for pattern, is_regex in replace_patterns:
            df_copy[columna] = df_copy[columna].str.replace(pattern, '', regex=is_regex)
        
        # Convert text to lowercase
        df_copy[columna] = df_copy[columna].str.lower()
    
    return df_copy


In [18]:
df_cleaned = clean_df(df_cleaned)

In [19]:
# Esto nos ayuda a saber si el signo o caracter esta todavia en la columna del dataset
contiene_caracter = df_cleaned['about_the_game'].astype(str).str.contains('•')

# Verifica si algún elemento contiene el carácter "®"
if contiene_caracter.any():
    print("La columna contiene al menos un '®'.")
else:
    print("La columna no contiene el carácter '®'.")

La columna no contiene el carácter '®'.


In [20]:
df_cleaned

,about_the_game,short_description,genres,supported_languages
0,for over two decades counterstrike has offered an elite competitive experience one shaped by millions of players from across the globe and now the next chapter in the cs story is about to begin this is counterstrike 2 a free upgrade to csgo counterstrike 2 marks the largest technical leap in counterstrike’s history built on the source 2 engine counterstrike 2 is modernized with realistic physicallybased rendering state of the art networking and upgraded community workshop tools in addition to the classic objectivefocused gameplay that counterstrike pioneered in 1999 counterstrike 2 features allnew cs ratings with the updated premier mode global and regional leaderboards upgraded and overhauled maps gamechanging dynamic smoke grenades tickrateindependent gameplay redesigned visual effects and audio all items from csgo moving forward to cs2,for over two decades counterstrike has offered an elite competitive experience one shaped by millions of players from across the globe and now the next chapter in the cs story is about to begin this is counterstrike 2,action free to play,czech danish dutch english finnish french german hungarian italian japanese korean norwegian polish portuguese portugal portuguese brazil romanian russian simplified chinese spanish spain swedish thai traditional chinese turkish bulgarian ukrainian greek spanish latin america vietnamese
1,land loot survive play pubg battlegrounds for free land on strategic locations loot weapons and supplies and survive to become the last team standing across various diverse battlegrounds squad up and join the battlegrounds for the original battle royale experience that only pubg battlegrounds can offer this content download will also provide access to the battlegrounds test server which requires a separate download to play optional ingame purchases available,play pubg battlegrounds for free land on strategic locations loot weapons and supplies and survive to become the last team standing across various diverse battlegrounds squad up and join the battlegrounds for the original battle royale experience that only pubg battlegrounds can offer,action adventure massively multiplayer free to play,english korean simplified chinese french german spanish spain arabic japanese polish portuguese portugal russian turkish thai italian portuguese brazil traditional chinese ukrainian
2,the mostplayed game on steam every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a constant evolution of gameplay features and heroes dota 2 has truly taken on a life of its own one battlefield infinite possibilities when it comes to diversity of heroes abilities and powerful items dota boasts an endless arrayno two games are the same any hero can fill multiple roles and theres an abundance of items to help meet the needs of each game dota doesnt provide limitations on how to play it empowers you to express your own style all heroes are free competitive balance is dotas crown jewel and to ensure everyone is playing on an even field the core content of the gamelike the vast pool of heroesis available to all players fans can collect cosmetics for heroes and fun addons for the world they inhabit but everything you need to play is already included before you join your first match bring your friends and party up dota is deep and constantly evolving but its never too late to join learn the ropes playing coop vs bots sharpen your skills in the hero demo mode jump into the behavior and skillbased matchmaking system that ensures youll be matched with the right players each game,every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a

In [21]:
df_cleaned.to_csv('df_cleaned_games.csv', index=False)

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Entrenamiento
</div>

In [22]:
nlp = spacy.load('en_core_web_sm')

In [23]:
def extraer_terminos_clave(texto):
    doc = nlp(texto)
    palabras_filtradas = [token.text for token in doc if not token.is_stop and token.is_alpha]
    return palabras_filtradas

In [24]:
palabras = []

for chunk in pd.read_csv('games_may2024_cleaned.csv', chunksize=1000):
    for texto in chunk['about_the_game']:
        if pd.notna(texto):
            palabra = extraer_terminos_clave((str)(texto))
            palabras.extend(palabra)


In [25]:
palabras_unidas = ' '.join(palabras)

In [26]:
nlp.max_length = 1500000  # Aumenta el límite de caracteres permitido

# Función para dividir el texto en fragmentos manejables
def dividir_texto(texto, n=100000):
    """Divide el texto en fragmentos de n caracteres."""
    for i in range(0, len(texto), n):
        yield texto[i:i + n]

In [27]:
# Divide el texto en fragmentos más pequeños
fragmentos = list(dividir_texto(palabras_unidas, n=100000))

# Inicializa un Counter para los términos clave
contador_terminos = Counter()

# Procesa cada fragmento y extrae términos clave con textacy
for fragmento in fragmentos:
    doc = textacy.make_spacy_doc(fragmento, lang='en_core_web_lg')
    keyterms = textacy.extract.keyterms.textrank(doc, topn=100)
    contador_terminos.update([term for term, _ in keyterms])

# Extrae los términos clave más comunes
terminos_clave = contador_terminos.most_common(10)

# Imprime los términos clave
print("Términos clave más comunes:", terminos_clave)

Términos clave más comunes: [('specific genre storydriven point amp click adventure game player', 205), ('game classic puzzle game level difficulty beautiful art character', 62), ('gameplay hd quality art beautiful girl pleasant atmosphere game mode easy medium advanced', 57), ('puzzle lovely girl incredible atmosphere great music lot level girls art hd quality game great', 54), ('game classic memory game level difficulty beautiful art character', 40), ('mouse game level game object character', 35), ('object puzzle adventure game', 29), ('object puzzle adventure game special collector edition release exclusive extra', 26), ('new game mode', 20), ('complete town country game huge puzzle town little piece', 17)]


In [28]:
# Guardar los términos clave en un archivo JSON
import json
with open('terminos_clave.json', 'w', encoding='utf-8') as file:
    json.dump(terminos_clave, file, ensure_ascii=False, indent=4)

In [29]:
# sentences = textacy.make_spacy_doc(palabras_unidas, lang='en_core_web_lg')
# keyterms=textacy.extract.keyterms.textrank(sentences,topn=100)

In [38]:
df_cleaned.columns

Index(['about_the_game', 'short_description', 'genres', 'supported_languages'], dtype='object')

In [31]:
# # Función para convertir el dataframe a un formato compatible con spaCy
# def df_to_spacy_format(df):
#     spacy_format = []
#     for _, row in df.iterrows():
#         spacy_format.append(row['about_the_game'])
#     return spacy_format

# # Convertir el dataframe a formato spaCy
# df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# # Guardar el dataframe en formato spaCy en un archivo de texto
# df_cleaned_file_path = 'df_cleaned_about_the_game.txt'
# with open(df_cleaned_file_path, 'w') as file:
#     for entry in df_cleaned_spacy_format:
#         file.write(entry + '\n')

# Función para convertir el dataframe a un formato compatible con spaCy
def df_to_spacy_format(df):
    spacy_format = []
    for _, row in df.iterrows():
        spacy_format.append(row['about_the_game'])
    return spacy_format

# Convertir el dataframe a formato spaCy
df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# Guardar el dataframe en formato spaCy en un archivo de texto
df_cleaned_file_path = 'df_cleaned_about_the_game.txt'
with open(df_cleaned_file_path, 'w', encoding='utf-8') as file:
    for entry in df_cleaned_spacy_format:
        file.write(entry + '\n')

In [32]:
# # Función para convertir el dataframe a un formato compatible con spaCy
# def df_to_spacy_format(df):
#     spacy_format = []
#     for _, row in df.iterrows():
#         spacy_format.append(row['short_description'])
#     return spacy_format

# # Convertir el dataframe a formato spaCy
# df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# # Guardar el dataframe en formato spaCy en un archivo de texto
# df_cleaned_file_path = 'df_cleaned_short_description.txt'
# with open(df_cleaned_file_path, 'w', encoding='utf-8') as file:
#     for entry in df_cleaned_spacy_format:
#         file.write(entry + '\n')

# Función para convertir el dataframe a un formato compatible con spaCy
def df_to_spacy_format(df):
    spacy_format = []
    for _, row in df.iterrows():
        spacy_format.append(row['short_description'])
    return spacy_format

# Convertir el dataframe a formato spaCy
df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# Guardar el dataframe en formato spaCy en un archivo de texto
df_cleaned_file_path = 'df_cleaned_short_description.txt'
with open(df_cleaned_file_path, 'w', encoding='utf-8') as file:
    for entry in df_cleaned_spacy_format:
        file.write(entry + '\n')

print(f"Datos guardados en {df_cleaned_file_path}")

Datos guardados en df_cleaned_short_description.txt


In [33]:
# # Función para convertir el dataframe a un formato compatible con spaCy
# def df_to_spacy_format(df):
#     spacy_format = []
#     for _, row in df.iterrows():
#         spacy_format.append(row['genres'])
#     return spacy_format

# # Convertir el dataframe a formato spaCy
# df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# # Guardar el dataframe en formato spaCy en un archivo de texto
# df_cleaned_file_path = 'df_cleaned_genres.txt'
# with open(df_cleaned_file_path, 'w') as file:
#     for entry in df_cleaned_spacy_format:
#         file.write(entry + '\n')

# Función para convertir el dataframe a un formato compatible con spaCy
def df_to_spacy_format(df):
    spacy_format = []
    for _, row in df.iterrows():
        spacy_format.append(row['genres'])
    return spacy_format

# Convertir el dataframe a formato spaCy
df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# Guardar el dataframe en formato spaCy en un archivo de texto
df_cleaned_file_path = 'df_cleaned_genres.txt'
with open(df_cleaned_file_path, 'w') as file:
    for entry in df_cleaned_spacy_format:
        file.write(entry + '\n')

print(f"Datos guardados en {df_cleaned_file_path}")

Datos guardados en df_cleaned_genres.txt


In [34]:
# Función para convertir el dataframe a un formato compatible con spaCy
# def df_to_spacy_format(df):
#     spacy_format = []
#     for _, row in df.iterrows():
#         spacy_format.append(row['supported_languages'])
#     return spacy_format

# # Convertir el dataframe a formato spaCy
# df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# # Guardar el dataframe en formato spaCy en un archivo de texto
# df_cleaned_file_path = 'df_supported_languages.txt'
# with open(df_cleaned_file_path, 'w') as file:
#     for entry in df_cleaned_spacy_format:
#         file.write(entry + '\n')

# Función para convertir el dataframe a un formato compatible con spaCy
def df_to_spacy_format(df):
    spacy_format = []
    for _, row in df.iterrows():
        spacy_format.append(row['supported_languages'])
    return spacy_format

# Convertir el dataframe a formato spaCy
df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# Guardar el dataframe en formato spaCy en un archivo de texto
df_cleaned_file_path = 'df_supported_languages.txt'
with open(df_cleaned_file_path, 'w', encoding='utf-8') as file:
    for entry in df_cleaned_spacy_format:
        file.write(entry + '\n')

print(f"Datos guardados en {df_cleaned_file_path}")

Datos guardados en df_supported_languages.txt


In [35]:
!python3 -m spacy download en_core_w


✘ No compatible package found for 'en_core_w' (spaCy v3.7.4)



In [36]:
import pandas as pd
import spacy

# Cargar el modelo de spaCy
nlp = spacy.load("en_core_web_sm")


# Función para procesar texto y extraer entidades
def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

# Cargar los archivos de texto en DataFrames
df_columna1 = pd.read_csv('df_cleaned_about_the_game.txt', header=None, names=['texto'])
df_columna2 = pd.read_csv('df_cleaned_genres.txt', header=None, names=['texto'])

# Procesar los textos y extraer entidades
df_columna1['entidades'] = df_columna1['texto'].apply(extraer_entidades)
df_columna2['entidades'] = df_columna2['texto'].apply(extraer_entidades)

# Mostrar los resultados
print(df_columna1)
print(df_columna2)

In [37]:
import spacy
from spacy.tokens import DocBin

# Cargar el modelo de spaCy
nlp = spacy.blank('en')

# Crear un DocBin para almacenar los ejemplos de entrenamiento
doc_bin = DocBin()

# Función para crear ejemplos de entrenamiento
def crear_ejemplo_ner(texto, entidades):
    doc = nlp.make_doc(texto)
    ents = []
    for start, end, label in entidades:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    doc.ents = ents
    doc_bin.add(doc)

# Ejemplo de datos etiquetados (texto y entidades)
ejemplos = [
    ("Grand Theft Auto V is a popular game.", [(0, 17, "GAME")]),
    ("Play PUBG Battlegrounds for free.", [(5, 21, "GAME")]),
]

# Crear ejemplos de entrenamiento
for texto, entidades in ejemplos:
    crear_ejemplo_ner(texto, entidades)

# Guardar los ejemplos en un archivo binario
doc_bin.to_disk("entrenamiento_ner.spacy")


TypeError: object of type 'NoneType' has no len()